# Code to scrape lost data

In [10]:
!pip install requests
!pip install io

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\tkbar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement io (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\tkbar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for io


In [12]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\tkbar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Scrape code

import pandas as pd
import requests
from bs4 import BeautifulSoup

# Base URL for lost satellites
base_url = "https://celestrak.org/satcat/lost.php"

# Initialize empty list to store data
all_rows = []

# Fetch the first page to determine pagination
response = requests.get(base_url)
if response.status_code != 200:
    print(f"Failed to retrieve data: {response.status_code}")
else:
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the total number of pages (if pagination exists)
    pagination = soup.find("div", class_="pagination")
    if pagination:
        pages = [a.text for a in pagination.find_all("a") if a.text.isdigit()]
        total_pages = max(map(int, pages)) if pages else 1
    else:
        total_pages = 1  # If no pagination, assume one page

    print(f"Total pages found: {total_pages}")

    # Loop through all pages
    for page in range(1, total_pages + 1):
        print(f"Scraping page {page}...")
        page_url = f"{base_url}?page={page}" if total_pages > 1 else base_url
        response = requests.get(page_url)
        
        if response.status_code != 200:
            print(f"Failed to retrieve page {page}: {response.status_code}")
            continue
        
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table")
        
        if table:
            # Extract headers from first page only
            if page == 1:
                headers = [header.text.strip() for header in table.find_all("th")]

            # Extract data rows
            for row in table.find_all("tr")[1:]:  # Skip header row
                cols = [col.text.strip() for col in row.find_all("td")]
                if cols:
                    all_rows.append(cols)

# Convert to DataFrame
df = pd.DataFrame(all_rows, columns=headers)

# Display the DataFrame
print(df.head())


Total pages found: 1
Scraping page 1...
  International Designator NORAD Catalog Number          Satellite Name  \
0                1960-009D                   52  ECHO 1 DEB [MYLAR OBJ]   
1                1963-039C                  692                  VELA 1   
2                1964-040A                  836       OPS 3662 (VELA 3)   
3                1965-058A                 1458       OPS 6577 (VELA 5)   
4                1965-058B                 1459       OPS 6564 (VELA 6)   

  Source Launch Date Launch Site Object Type                   Last Data  
0     US  1960-08-12       AFETR         DEB  1991-03-25T14:23:57.069312  
1     US  1963-10-17       AFETR        PAY-  2024-11-06T17:09:09.800928  
2     US  1964-07-17       AFETR        PAY-  2024-10-18T19:49:13.468512  
3     US  1965-07-20       AFETR        PAY-  2025-01-24T09:01:13.708704  
4     US  1965-07-20       AFETR        PAY-  2024-12-24T02:18:46.645344  


In [16]:
df.to_csv("CelesTrek/lost_objects.csv", index = False)